# Who is the best pokemon?

importing necessary libraries

In [1]:
from determine_job import DetermineJob
from generate_data import generate_random_pokemon
from simulate import simulate_round
import pandas as pd
import numpy as np
import requests
import os
import random

import warnings
warnings.filterwarnings("ignore")

Jack-Of-All-Trades


parse_json takes a string url as input and then returns a dict with info of the specific pokemon from the API

In [2]:
def parse_json(url, isInfo):
    response = requests.get(url)
    content = response.json()
    
    if isInfo:
        name = content['name']
        moveset = ', '.join([move['move']['name'] for move in content['moves']])
        types = ', '.join([p_type['type']['name'] for p_type in content['types']])
        stats = [stat['base_stat'] for stat in content['stats']]
        
        poke_info = {'NAME': name , 'TYPE': types, 'HP': stats[0], 'ATTACK': stats[1], 'DEFENSE': stats[2], 
                                    'SPECIAL-ATTACK': stats[3], 'SPECIAL-DEFENSE': stats[4], 'SPEED': stats[5], 
                                    'TOTAL': sum(stats), 'MOVESET': moveset}
    else:
        p_type_name = content['name']
        poke_info = {p_type_name: {relation: [name['name'] for name in content['damage_relations'][relation]] 
                                   for relation in content['damage_relations']}}
        

    return poke_info
        

run through every pokemon in the pokeDB API and then store them in a Dataframe called poke_df

In [3]:
#done through this method to lessen time to debug and not constantly deal with timeout on API

if not os.path.isfile('excel_files/poke_db.xlsx'):
    poke_df = pd.DataFrame(columns=['NAME', 'TYPE', 'HP', 'ATTACK', 'DEFENSE', 
                                'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED', 
                                'TOTAL', 'MOVESET'])

    poke_id = 1
    poke_id_secondary = 10001

    while True:
        try:
            pID = poke_id_secondary if poke_id > 1025 else poke_id
            poke_df = pd.concat([poke_df, pd.DataFrame([parse_json(f'https://pokeapi.co/api/v2/pokemon/{pID}/', True)])], ignore_index=True)

            if poke_id > 1025:
                poke_id_secondary += 1
            else:
                poke_id += 1
        except:
            break
        
    poke_df.to_excel('excel_files/poke_db.xlsx')
else:
    poke_df = pd.read_excel('excel_files/poke_db.xlsx')
    
poke_df = poke_df.drop('Unnamed: 0', axis=1)

In [4]:
poke_df

,NAME,TYPE,HP,ATTACK,DEFENSE,SPECIAL-ATTACK,SPECIAL-DEFENSE,SPEED,TOTAL,MOVESET
0,bulbasaur,"grass, poison",45,49,49,65,65,45,318,"razor-wind, swords-dance, cut, bind, vine-whip..."
1,ivysaur,"grass, poison",60,62,63,80,80,60,405,"swords-dance, cut, bind, vine-whip, headbutt, ..."
2,venusaur,"grass, poison",80,82,83,100,100,80,525,"swords-dance, cut, bind, vine-whip, headbutt, ..."
3,charmander,fire,39,52,43,60,50,65,309,"mega-punch, fire-punch, thunder-punch, scratch..."
4,charmeleon,fire,58,64,58,80,65,80,405,"mega-punch, fire-punch, thunder-punch, scratch..."
...,...,...,...,...,...,...,...,...,...,...
1297,ogerpon-wellspring-mask,"grass, water",80,120,84,60,96,110,550,"swords-dance, slam, vine-whip, double-kick, ta..."
1298,ogerpon-hearthflame-mask,"grass, fire",80,120,84,60,96,110,550,"swords-dance, slam, vine-whip, double-kick, ta..."
1299,ogerpon-cornerstone-mask,"grass, rock",80,120,84,60,96,110,550,"swords-dance, slam, vine-whip, double-kick, ta..."
1300,terapagos-terastal,normal,95,95,110,105,110,85,600,NaN


get the type advantages and input them into a dictionary and excel file to handle simulation of matches

In [5]:
if not os.path.isfile('excel_files/type_advantages.xlsx'):
    type_id = 1
    types = dict()
    
    while True:
        try:
            types.update(parse_json(f"https://pokeapi.co/api/v2/type/{type_id}", False))
            type_id += 1
        except:
            break
        
    move_adv_df = pd.DataFrame.from_dict(types)
    move_adv_df.to_excel('excel_files/type_advantages.xlsx')
else:
    move_adv_df = pd.read_excel('excel_files/type_advantages.xlsx')

In [6]:
move_adv_df

,Unnamed: 0,normal,fighting,flying,poison,ground,rock,bug,ghost,steel,fire,water,grass,electric,psychic,ice,dragon,dark,fairy
0,double_damage_from,['fighting'],"['flying', 'psychic', 'fairy']","['rock', 'electric', 'ice']","['ground', 'psychic']","['water', 'grass', 'ice']","['fighting', 'ground', 'steel', 'water', 'grass']","['flying', 'rock', 'fire']","['ghost', 'dark']","['fighting', 'ground', 'fire']","['ground', 'rock', 'water']","['grass', 'electric']","['flying', 'poison', 'bug', 'fire', 'ice']",['ground'],"['bug', 'ghost', 'dark']","['fighting', 'rock', 'steel', 'fire']","['ice', 'dragon', 'fairy']","['fighting', 'bug', 'fairy']","['poison', 'steel']"
1,double_damage_to,[],"['normal', 'rock', 'steel', 'ice', 'dark']","['fighting', 'bug', 'grass']","['grass', 'fairy']","['poison', 'rock', 'steel', 'fire', 'electric']","['flying', 'bug', 'fire', 'ice']","['grass', 'psychic', 'dark']","['ghost', 'psychic']","['rock', 'ice', 'fairy']","['bug', 'steel', 'grass', 'ice']","['ground', 'rock', 'fire']","['ground', 'rock', 'water']","['flying', 'water']","['fighting', 'poison']","['flying', 'ground', 'grass', 'dragon']",['dragon'],"['ghost', 'psychic']","['fighting', 'dragon', 'dark']"
2,half_damage_from,[],"['rock', 'bug', 'dark']","['fighting', 'bug', 'grass']","['fighting', 'poison', 'bug', 'grass', 'fairy']","['poison', 'rock']","['normal', 'flying', 'poison', 'fire']","['fighting', 'ground', 'grass']","['poison', 'bug']","['normal', 'flying', 'rock', 'bug', 'steel', '...","['bug', 'steel', 'fire', 'grass', 'ice', 'fairy']","['steel', 'fire', 'water', 'ice']","['ground', 'water', 'grass', 'electric']","['flying', 'steel', 'electric']","['fighting', 'psychic']",['ice'],"['fire', 'water', 'grass', 'electric']","['ghost', 'dark']","['fighting', 'bug', 'dark']"
3,half_damage_to,"['rock', 'steel']","['flying', 'poison', 'bug', 'psychic', 'fairy']","['rock', 'steel', 'electric']","['poison', 'ground', 'rock', 'ghost']","['bug', 'grass']","['fighting', 'ground', 'steel']","['fighting', 'flying', 'poison', 'ghost', 'ste...",['dark'],"['steel', 'fire', 'water', 'electric']","['rock', 'fire', 'water', 'dragon']","['water', 'grass', 'dragon']","['flying', 'poison', 'bug', 'steel', 'fire', '...","['grass', 'electric', 'dragon']","['steel', 'psychic']","['steel', 'fire', 'water', 'ice']",['steel'],"['fighting', 'dark', 'fairy']","['poison', 'steel', 'fire']"
4,no_damage_from,['ghost'],[],['ground'],[],['electric'],[],[],"['normal', 'fighting']",['poison'],[],[],[],[],[],[],[],['psychic'],['dragon']
5,no_damage_to,['ghost'],['ghost'],[],['steel'],['flying'],[],[],['normal'],[],[],[],[],['ground'],['dark'],[],['fairy'],[],[]


The DetermineJob class has multiple methods, the method that is used is get_jobs, it takes input an array of stats and their moveset, it outputs an array of their jobs

In [7]:
job_determine = DetermineJob()

for i in range(100):
    print(poke_df.iloc[i]['NAME'], job_determine.get_jobs_primary([poke_df.iloc[i]['HP'], poke_df.iloc[i]['ATTACK'], poke_df.iloc[i]['DEFENSE'], 
                           poke_df.iloc[i]['SPECIAL-ATTACK'], poke_df.iloc[i]['SPECIAL-DEFENSE'], poke_df.iloc[i]['SPEED']]))
    print(job_determine.get_jobs_secondary(poke_df.iloc[i]['MOVESET']))

bulbasaur Special Tank
['Staller', 'Toxi-Shufflers']
ivysaur Special Tank
['Staller', 'Toxi-Shufflers']
venusaur Jack-Of-All-Trades
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
charmander Special Sweeper
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
charmeleon Special Sweeper
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
charizard Special Sweeper
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
squirtle Physical Tank
['Hazer', 'Staller', 'Toxi-Shufflers']
wartortle Physical Tank
['Staller', 'Toxi-Shufflers']
blastoise Jack-Of-All-Trades
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers']
caterpie Physical Tank
[]
metapod Physical Tank
[]
butterfree Special Sweeper
['Pseudo-Hazer', 'Staller', 'Toxi-Shufflers', 'Para-Shufflers', 'Pivots']
weedle Physical Sweeper
[]
kakuna Physical Tank
[]
beedrill Physical Sweeper
['Staller', 'Toxi-Shufflers', 'Pivots']
pidgey Physical Sweeper
['Pseudo-Hazer', 'Staller', 'T

Creating a dataset consisting of 10,000 random pokemons which will be assinged jobs, once they are assinged jobs, the dataset will be used to predict the jobs of the actual pokemon

In [8]:
pokemon_df = generate_random_pokemon(100000)

In [9]:
pokemon_df

,HP,Attack,Defense,Special-Attack,Special-Defense,Speed,Job
0,123,119,166,185,91,64,Jack-Of-All-Trades
1,178,5,222,98,169,114,Jack-Of-All-Trades
2,35,52,5,105,64,68,Special Sweeper
3,82,87,109,16,79,155,Physical Sweeper
4,210,124,195,67,204,30,Jack-Of-All-Trades
...,...,...,...,...,...,...,...
99995,140,77,223,172,167,69,Jack-Of-All-Trades
99996,118,69,126,141,223,5,Jack-Of-All-Trades
99997,8,124,208,176,74,147,Jack-Of-All-Trades
99998,58,148,38,49,111,97,Physical Sweeper


using the pokemon_df dataset created to predict the jobs of actual pokemons

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

pokemon_df['Job'] = pokemon_df['Job'].astype('category')

X_train, X_test, y_train, y_test = train_test_split(pokemon_df.drop(['Job'], axis='columns'), pokemon_df.Job, test_size=0.2)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9584

Assinging the primary job to the pokemons, this is only stats based

Also assinging secondary jobs which is based on movesets

In [11]:
poke_df['Primary Job'] = None
poke_df['Secondary Job'] = None

for index, row in poke_df.iterrows():
    stats = [poke_df.at[index, stat.upper()] for stat in ['HP', 'ATTACK', 'DEFENSE', 
                                'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED',]]
    moveset = poke_df.at[index, 'MOVESET']
    if type(moveset) != float:
        poke_df.at[index, 'Secondary Job'] = (', ').join(job_determine.get_jobs_secondary(moveset))
    poke_df.at[index, 'Primary Job'] = clf.predict([stats])

In [12]:
poke_df

,NAME,TYPE,HP,ATTACK,DEFENSE,SPECIAL-ATTACK,SPECIAL-DEFENSE,SPEED,TOTAL,MOVESET,Primary Job,Secondary Job
0,bulbasaur,"grass, poison",45,49,49,65,65,45,318,"razor-wind, swords-dance, cut, bind, vine-whip...",Special Sweeper,"Staller, Toxi-Shufflers"
1,ivysaur,"grass, poison",60,62,63,80,80,60,405,"swords-dance, cut, bind, vine-whip, headbutt, ...",Special Sweeper,"Staller, Toxi-Shufflers"
2,venusaur,"grass, poison",80,82,83,100,100,80,525,"swords-dance, cut, bind, vine-whip, headbutt, ...",Jack-Of-All-Trades,"Pseudo-Hazer, Staller, Toxi-Shufflers, Para-Sh..."
3,charmander,fire,39,52,43,60,50,65,309,"mega-punch, fire-punch, thunder-punch, scratch...",Special Sweeper,"Pseudo-Hazer, Staller, Toxi-Shufflers, Para-Sh..."
4,charmeleon,fire,58,64,58,80,65,80,405,"mega-punch, fire-punch, thunder-punch, scratch...",Special Sweeper,"Pseudo-Hazer, Staller, Toxi-Shufflers, Para-Sh..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1297,ogerpon-wellspring-mask,"grass, water",80,120,84,60,96,110,550,"swords-dance, slam, vine-whip, double-kick, ta...",Physical Sweeper,"Staller, Spiker, Pivots"
1298,ogerpon-hearthflame-mask,"grass, fire",80,120,84,60,96,110,550,"swords-dance, slam, vine-whip, double-kick, ta...",Physical Sweeper,"Staller, Spiker, Pivots"
1299,ogerpon-cornerstone-mask,"grass, rock",80,120,84,60,96,110,550,"swords-dance, slam, vine-whip, double-kick, ta...",Physical Sweeper,"Staller, Spiker, Pivots"
1300,terapagos-terastal,normal,95,95,110,105,110,85,600,NaN,Jack-Of-All-Trades,None


In [13]:
poke_df['Primary ELO'] = poke_df['TOTAL']

elo_changes = {idx: 0 for idx in poke_df.index}

for index, row in poke_df.iterrows():
    poke1 = {
        'name': row['NAME'],
        'stats': [row[stat] for stat in ['HP', 'ATTACK', 'DEFENSE', 'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED']],
        'type': [t.strip() for t in row['TYPE'].split(',')],
        'Primary ELO': row['Primary ELO']
    }

    # Rate against every other pokemon
    for index2, row2 in poke_df.iterrows():
        if index < index2:  
            poke2 = {
                'name': row2['NAME'],
                'stats': [row2[stat] for stat in ['HP', 'ATTACK', 'DEFENSE', 'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED']],
                'type': [t.strip() for t in row2['TYPE'].split(',')],
                'Primary ELO': row2['Primary ELO']
            }

            new_elo1, new_elo2 = simulate_round(poke1, poke2, move_adv_df)
            elo_changes[index] += new_elo1 
            elo_changes[index2] += new_elo2 

for idx, change in elo_changes.items():
    poke_df.at[idx, 'Primary ELO'] += change


In [17]:
poke_df.to_excel('ratings.xlsx')